# What is AutoGluon?

AutoGluon is a new __open source AutoML library that automates deep learning (DL) and machine learning (ML)__ for real world applications involving image, text and tabular datasets. With AutoGluon, you can develop and refine state-of-the-art DL models using just a few lines of Python code.

# Motivation
- Historically, achieving state-of-the-art ML performance required extensive background knowledge, experience, and human effort. 
- Data preparation, feature engineering, validation splitting, missing value handling, and model selection are just a few of the many tasks that must be addressed in ML applications. One particularly difficult task is the selection of hyperparameters.
- Hyperparameters represent the many choices that must be made by the user when constructing a model, such as the data processing steps, neural network architecture, and the optimizer used during training. 

# Key Features

- AutoGluon will leverage the available compute resources to find the strongest ML methods within its allotted run-time.
- AutoGluon enables you to automatically __achieve state-of-the-art performance on tasks such as image classification, object detection, text classification, and supervised learning with tabular datasets.__ 
- __The hyperparameters of each task are automatically selected using advanced tuning algorithms such as Bayesian Optimization, Hyperband,__ 
- Reinforcement Learning. With AutoGluon, you don’t have to have any familiarity with the underlying models, as all hyperparameters will be automatically tuned within default ranges that are known to perform well for the particular task and model.
- For expert ML practitioners, AutoGluon allows this process to be easily customized. For example, you can specify ranges of values to consider for certain hyperparameters, and also use AutoGluon to automatically tune various aspects of your own custom models. 
- If you have access to multiple machines, AutoGluon can easily distribute its computation across them in order to return trained models more quickly.

# Installation
- CUDA 10.0 and a GPU for object detection is recommended
- autogluon have dependency of mxnet

pip install --upgrade mxnet-cu100\
pip install autogluon

# Implementation
>__Predicting Columns in a Table - In Depth__

Start by importing AutoGluon, specifying TabularPrediction as the task, and loading the data.

In [3]:
import autogluon as ag
from autogluon import TabularPrediction as task

In [45]:
train_data = task.Dataset(file_path='https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
#train_data = train_data.sample(500)

Loaded data from: https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv | Columns = 15 / 15 | Rows = 39073 -> 39073


In [10]:
train_data.shape

(39073, 15)

In [17]:
train_data.head(1)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,178478,Bachelors,13,Never-married,Tech-support,Own-child,White,Female,0,0,40,United-States,<=50K


If you do not wish to provide a validation dataset, __it is recommended that you omit the tuning_data argument. This lets AutoGluon automatically select validation data from your provided training set__ (it uses smart strategies such as stratified sampling). For greater control, you can specify the holdout_frac argument to tell AutoGluon what fraction of the provided training data to hold out for validation.

In [13]:
label_column = 'occupation'
val_data = task.Dataset(file_path='https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
print("Summary of occupation column: \n", train_data['occupation'].describe())

Summary of occupation column: 
 count               39073
unique                 15
top        Prof-specialty
freq                 4949
Name: occupation, dtype: object


In [43]:
import pandas as pd
pd.unique(train_data[label_column])

array([' Exec-managerial', ' Craft-repair', ' Handlers-cleaners',
       ' Adm-clerical', ' Other-service', ' Sales', ' ?',
       ' Protective-serv', ' Farming-fishing', ' Transport-moving',
       ' Prof-specialty', ' Machine-op-inspct', ' Tech-support'],
      dtype=object)

In [48]:
# whether or not to do hyperparameter optimization
hp_tune = True

# specifies non-default hyperparameter values for neural network models
nn_options = {
    'num_epochs': 100, # number of training epochs (controls training time of NN models)
    'learning_rate': ag.space.Real(1e-4, 1e-2, default=5e-4, log=True), # learning rate used in training (real-valued hyperparameter searched on log-scale)
    'activation': ag.space.Categorical('relu', 'softrelu', 'tanh'), # activation function used in NN (categorical hyperparameter, default = first entry)
    'layers': ag.space.Categorical([100],[1000],[200,100],[300,200,100]),
      # Each choice for categorical hyperparameter 'layers' corresponds to list of sizes for each NN layer to use
    'dropout_prob': ag.space.Real(0.0, 0.5, default=0.1), # dropout probability (real-valued hyperparameter)
}

gbm_options = { # specifies non-default hyperparameter values for lightGBM gradient boosted trees
    'num_boost_round': 100, # number of boosting rounds (controls training time of GBM models)
    'num_leaves': ag.space.Int(lower=26, upper=66, default=36), # number of leaves in trees (integer hyperparameter)
}

hyperparameters = {'NN': nn_options, 'GBM': gbm_options}  # hyperparameters of each model type
# If one of these keys is missing from hyperparameters dict, then no models of that type are trained.

time_limits = 2*60  # train various models for ~2 min
num_trials = 5  # try at most 3 different hyperparameter configurations for each type of model
search_strategy = 'skopt'  # to tune hyperparameters using SKopt Bayesian optimization routine
output_directory = 'output'  # folder where to store trained models

two other methods to boost predictive performance are bagging and stack-ensembling. You’ll often see performance improve if you specify num_bagging_folds = 5-10, stack_ensemble_levels = 1 or 2 in the call to fit(), but this will increase training times.

- num_bagging_folds : (int)\
Number of folds used for bagging of models. When `num_bagging_folds = k`, training time is roughly increased by a factor of `k` (set = 0 to disable bagging).\
Disabled by default, but we recommend values between 5-10 to maximize predictive performance. 
    
- stack_ensemble_levels : (int)\
Number of stacking levels to use in stack ensemble. Roughly increases model training time by factor of `stack_ensemble_levels+1` (set = 0 to disable stack ensembling).

In [49]:
predictor = task.fit(train_data=train_data, tuning_data=val_data, label=label_column,
                     output_directory=output_directory, time_limits=time_limits, num_trials=num_trials,
                     hyperparameter_tune=hp_tune, hyperparameters=hyperparameters,
                     search_strategy=search_strategy, num_bagging_folds=5, stack_ensemble_levels=1)

Beginning AutoGluon training ... Time limit = 120s
Preprocessing data ...
Here are the first 10 unique label values in your data:  [' Tech-support' ' Transport-moving' ' Other-service' ' ?'
 ' Handlers-cleaners' ' Sales' ' Craft-repair' ' Adm-clerical'
 ' Exec-managerial' ' Prof-specialty']
AutoGluon infers your prediction problem is: multiclass  (because dtype of label-column == object)
If this is wrong, please specify `problem_type` argument in fit() instead (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])

	Data preprocessing and feature engineering runtime = 0.19s ...
AutoGluon will gauge predictive performance using evaluation metric: accuracy
To change this, specify the eval_metric argument of fit()
Fitting model: LightGBMClassifier ... Training model for up to 59.9s of the 119.81s of remaining time.
Attempting to fit model without HPO, but search space is provided. fit() will only consider default hyperparameter values from search space.
	Ran out 

We again demonstrate how to use the trained models to predict on the validation data. <span class="burk">We caution again that performance estimates from this data may be biased because the same  data was used to tune hyperparameters.</span>

In [50]:
test_data = val_data.copy()
y_test = test_data[label_column]
test_data.drop(labels=[label_column],axis=1, inplace = True)

In [51]:
np.sum(test_data.columns == label_column)

0

In [52]:
test_data.sample(1)

,age,workclass,fnlwgt,education,education-num,marital-status,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
2111,19,Local-gov,27382,Some-college,10,Never-married,Own-child,White,Male,0,0,40,United-States,<=50K


In [53]:
y_pred = predictor.predict(test_data)

print("Predictions:  ", list(y_pred)[:5])
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: accuracy on test data: 0.424813
Evaluations on test data:
{
    "accuracy": 0.4248131845634149,
    "accuracy_score": 0.4248131845634149,
    "balanced_accuracy_score": 0.40748268871019905,
    "matthews_corrcoef": 0.3603755551444519
}


Predictions:   [' Other-service', ' Farming-fishing', ' Exec-managerial', ' Other-service', ' Other-service']


Detailed (per-class) classification report:
{
    " ?": {
        "precision": 1.0,
        "recall": 0.99830220713073,
        "f1-score": 0.9991503823279524,
        "support": 589
    },
    " Adm-clerical": {
        "precision": 0.39043309631544926,
        "recall": 0.5206896551724138,
        "f1-score": 0.44625046176579247,
        "support": 1160
    },
    " Armed-Forces": {
        "precision": 0.2,
        "recall": 1.0,
        "f1-score": 0.33333333333333337,
        "support": 2
    },
    " Craft-repair": {
        "precision": 0.3275186899252403,
        "recall": 0.7204385277995301,
        "f1-score": 0.45031815956926086,
        "support": 1277
    },
    " Exec-managerial": {
        "precision": 0.39718804920913886,
        "recall": 0.36044657097288674,
        "f1-score": 0.37792642140468224,
        "support": 1254
    },
    " Farming-fishing": {
        "precision": 0.45121951219512196,
        "recall": 0.2868217054263566,
        "f1-score": 0.3507109004739

predictor can also make a prediction on an individual example rather than a full dataset

In [54]:
datapoint = test_data.sample(1)  # Note: .iloc[0] won't work because it returns pandas Series instead of DataFrame
datapoint

,age,workclass,fnlwgt,education,education-num,marital-status,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
8911,75,Self-emp-not-inc,192813,Masters,14,Widowed,Not-in-family,White,Male,0,0,45,United-States,<=50K


In [55]:
print("Prediction:", predictor.predict(datapoint))

Prediction: [' Prof-specialty']


Use the following to view a summary of what happened during fit. This command will shows details of the hyperparameter-tuning process for each type of model:

In [56]:
results = predictor.fit_summary()

*** Summary of fit() ***
Number of models trained: 4
Types of models trained: 
{'WeightedEnsembleModel', 'StackerEnsembleModel', 'BaggedEnsembleModel'}
Validation performance of individual models: {'LightGBMClassifier_BAGGED_l0': 0.3945170140452889, 'weighted_ensemble_l1': 0.3945170140452889, 'LightGBMClassifier_STACKER_l1': 0.38833381106424797, 'weighted_ensemble_l2': 0.38833381106424797}
Best model (based on validation performance): weighted_ensemble_l1
Hyperparameter-tuning used: False
Bagging used: True  (with 5 folds)
Stack-ensembling used: True  (with 1 levels)
User-specified hyperparameters:
{'NN': {'num_epochs': 100, 'learning_rate': Real: lower=0.0001, upper=0.01, 'activation': Categorical['relu', 'softrelu', 'tanh'], 'layers': Categorical[[100], [1000], [200, 100], [300, 200, 100]], 'dropout_prob': Real: lower=0.0, upper=0.5}, 'GBM': {'num_boost_round': 100, 'num_leaves': Int: lower=26, upper=66}}
Plot summary of models saved to file: SummaryOfModels.html
*** End of fit() sum

/home/labs/anaconda3/envs/deploy/lib/python3.6/site-packages/autogluon/utils/plots.py:133: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. Please do: "pip install bokeh"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. Please do: "pip install bokeh"')
